In [10]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [4]:
import tweepy
import time
import json
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import re





In [2]:
!pip install sentence_transformers
!pip install emoji

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [72]:
X_train = []
X_dev = []
train_ids = open("drive/MyDrive/NLP_dataset/train.data.txt", 'r').readlines()
dev_ids = open("drive/MyDrive/NLP_dataset/dev.data.txt", 'r').readlines()
test_ids = open("drive/MyDrive/NLP_dataset/test.data.txt", 'r').readlines()
data = json.load(open("drive/MyDrive/NLP_dataset/data.txt"))
data_dev = json.load(open("drive/MyDrive/NLP_dataset/data_dev.txt"))
y_train_t = open('drive/MyDrive/NLP_dataset/train.label.txt','r').readlines()
y_dev_t = open('drive/MyDrive/NLP_dataset/dev.label.txt','r').readlines()



In [5]:

y_train_t = open('train.label.txt','r').readlines()
y_dev_t = open('dev.label.txt','r').readlines()
train_ids = open("train.data.txt", 'r').readlines()
dev_ids = open("dev.data.txt", 'r').readlines()
test_ids = open("test.data.txt", 'r').readlines()
data = json.load(open("data.txt"))
data_dev = json.load(open("data_dev.txt"))
X_train = []
X_dev = []

In [6]:
import time
tra_data = []
tra_data2 = []

y_train = []
train_additional = []
for i in range(len(train_ids)):
    ids = train_ids[i]
    ids = ids.strip().split(",")
    temp = []
    
    for j in range(len(ids)):
        ele = ids[j]
        for ele2 in data[i]:
            if ele == str(ele2['id']):
                temp.append(ele2)
        if j == 0 and temp == []:
            break
    if temp == []:
        continue        
    temp2 = sorted(list(temp[1:]) , key = lambda x : time.mktime(time.strptime(x['created_at'], "%a %b %d %H:%M:%S %z %Y")))
    temp3 = [temp[0]] + temp2
    temp3 = list(map(lambda x : x['text'],temp3)) 
    tra_data += temp3
    tra_data2.append(temp3)
    label = y_train_t[i]
    train_additional.append(
        [int(temp[0]['user']['followers_count']),
         int(temp[0]['user']['friends_count']),
         int(temp[0]['user']['statuses_count']),
         int(temp[0]['user']['favourites_count']),
         int(temp[0]['user']['verified']),
         float(time.mktime(time.strptime(temp[0]['created_at'], "%a %b %d %H:%M:%S %z %Y"))
         -time.mktime(time.strptime(temp[0]['user']['created_at'], "%a %b %d %H:%M:%S %z %Y")))/31536000
         ])
    if label == 'nonrumour\n':
        y_train.append(0)
    else:
        y_train.append(1)

dev_data = []
dev_data2 = []
y_dev = []
dev_additional = []
for i in range(len(dev_ids)):
    ids = dev_ids[i]
    ids = ids.strip().split(",")
    temp = []
    for j in range(len(ids)):
        ele = ids[j]
        for ele2 in data_dev[i]:
            if ele == str(ele2['id']):
                temp.append(ele2)
        if j == 0 and temp == []:
            break
    if temp == []:
        continue
    temp2 = sorted(temp[1:] , key = lambda x : time.mktime(time.strptime(x['created_at'], "%a %b %d %H:%M:%S %z %Y")))
    temp3 = [temp[0]] + temp2
    temp3 = list(map(lambda x : x['text'],temp3)) 
    dev_data += temp3
    dev_data2.append(temp3)

    dev_additional.append(
        [int(temp[0]['user']['followers_count']),
         int(temp[0]['user']['friends_count']),
         int(temp[0]['user']['statuses_count']),
         int(temp[0]['user']['favourites_count']),
         int(temp[0]['user']['verified']),
         float(time.mktime(time.strptime(temp[0]['created_at'], "%a %b %d %H:%M:%S %z %Y"))
         -time.mktime(time.strptime(temp[0]['user']['created_at'], "%a %b %d %H:%M:%S %z %Y")))/31536000
         ])
    
    label = y_dev_t[i]
    if label == 'nonrumour\n':
        y_dev.append(0)
    else:
        y_dev.append(1)

In [7]:
tt = TweetTokenizer()
#stopwords = set(stopwords.words('english'))

def preprocess(lst):
    temp = []
    #list of tweets
    for tweet in lst:
        tweet = tweet.strip().split(" ")
        

        tweet = list(map(lambda x: x.lower(), tweet))
        #tweet = list(map(lambda x: x.isalpha(), tweet))
        #remove any word that does not contain any English alphabets
        #tweet = list(map(lambda x: re.sub("^[^a-z\d]+$", "", x).strip(), tweet))
        tweet = list(map(lambda x: re.sub("^(http).*$", "", x).strip(), tweet))
        #tweet = list(map(lambda x: re.sub("[^\w\s@#]", "", x).strip(), tweet))
        tweet = list(map(lambda x: re.sub("^[@].*$", "", x).strip(), tweet))
        tweet = list(map(lambda x: re.sub("\n", " ", x).strip(), tweet))
        #remove all '' in the list
        tweet = list(filter(lambda x: x != 'rt', tweet))
        tweet = list(filter(lambda x: x != '', tweet))
        #tweet = list(filter(lambda x: x not in stopwords, tweet))
        tweet = ' '.join(tweet)
        temp.append(tweet)
    return temp


text_data = preprocess(tra_data)
print(text_data[1])
text_dev_data = preprocess(dev_data)
print(text_dev_data[0])

# test_id = open("drive/MyDrive/NLP_dataset/test.data.txt", 'r').readlines()
# test_data = []
# test_additional = []
# for i in range(len(test_id)):
#     ids = test_id[i]
#     ids = ids.strip().split(",")
#     temp = []
#     for ele in ids:
#         temp.append(json.load(open("drive/MyDrive/NLP_dataset/test-tweet-objects/" + str(ele) + ".json")))
#     temp2 = sorted(temp[1:] , key = lambda x : time.mktime(time.strptime(x['created_at'], "%a %b %d %H:%M:%S %z %Y")))
#     temp3 = [temp[0]] + temp2
    
#     test_additional.append(
#         [int(temp[0]['user']['followers_count']),
#          int(temp[0]['user']['friends_count']),
#          int(temp[0]['user']['statuses_count']),
#          int(temp[0]['user']['favourites_count']),
#          int(temp[0]['user']['verified']),
#          float(time.mktime(time.strptime(temp[0]['created_at'], "%a %b %d %H:%M:%S %z %Y"))-time.mktime(time.strptime(temp[0]['user']['created_at'], "%a %b %d %H:%M:%S %z %Y")))/31536000
#          ])
#     # temp4 = list(map(lambda x : x['text'],temp3)) 
#     test_data.append(temp3)
    
# text_test_data = []
# for ele in test_data:
#     temp = list(map(lambda x : x['text'],ele))
#     temp = preprocess(temp)
#     text_test_data += temp

4. can eating garlic help prevent infection with the new coronavirus? #covid19malaysia
covid-19 fact: are hand dryers effective in killing the new coronavirus?  share the information with your loved one…


In [137]:
list(map(lambda x: re.sub("RT", "", x).strip(), ['RT RT RTT']))

['T']

In [8]:
test_id = open("test.data.txt", 'r').readlines()
test_data = []
for i in range(len(test_id)):
    ids = test_id[i]
    ids = ids.strip().split(",")
    temp = []
    for ele in ids:
        temp.append(json.load(open("tweet-objects/tweet-objects/" + str(ele) + ".json")))
    temp2 = sorted(temp[1:] , key = lambda x : time.mktime(time.strptime(x['created_at'], "%a %b %d %H:%M:%S %z %Y")))
    temp3 = [temp[0]] + temp2
    #temp3 = list(map(lambda x : x['text'],temp3)) 
    test_data.append(temp3)
    


In [138]:
test_data = json.load(open("drive/MyDrive/NLP_dataset/data_test.txt"))

In [9]:
test_additional = []
text_test_data = []
y_test = []
for temp in test_data:
    test_additional.append(
        [int(temp[0]['user']['followers_count']),
         int(temp[0]['user']['friends_count']),
         int(temp[0]['user']['statuses_count']),
         int(temp[0]['user']['favourites_count']),
         int(temp[0]['user']['verified']),
         float(time.mktime(time.strptime(temp[0]['created_at'], "%a %b %d %H:%M:%S %z %Y"))-time.mktime(time.strptime(temp[0]['user']['created_at'], "%a %b %d %H:%M:%S %z %Y")))/31536000
         ])
    temp1 = list(map(lambda x : x['text'],temp))
    temp1 = preprocess(temp1)
    text_test_data += temp1

# for i in range(len(test_ids)):
#     ids = test_ids[i]
#     ids = ids.strip().split(",")
#     temp = []
#     for j in range(len(ids)):
#         ele = ids[j]
#         for ele2 in test_data[i]:
#             if ele == str(ele2['id']):
#                 temp.append(ele2)
#         if j == 0 and temp == []:
#             break
#     if temp == []:
#         continue
    
#     label = y_test_t[i]
#     if label == 'nonrumour\n':
#         y_test.append(0)
#     else:
#         y_test.append(1)

In [10]:
y_train = np.array(y_train)
y_dev = np.array(y_dev)

train_additional = np.array(train_additional)
dev_additional = np.array(dev_additional)
test_additional = np.array(test_additional)

In [14]:
len(text_test_data)

7996

In [11]:
def rebuild(embeddings,file):
    lst = []
    idx = 0
    for i in range(len(file)):
        temp = []
        ids = file[i]
        for ele in ids:
            temp.append(embeddings[idx])
            idx+=1
        lst.append(temp)
    return lst
train_data_rebuilt = rebuild(text_data,tra_data2)
dev_data_rebuilt = rebuild(text_dev_data,dev_data2)
test_data_rebuilt = rebuild(text_test_data,test_data)

In [16]:
train_data_rebuilt[0]


['5. can regularly rinsing your nose with saline help prevent infection with the new coronavirus?',
 '4. can eating garlic help prevent infection with the new coronavirus? #covid19malaysia',
 '6. do vaccines against pneumonia protect you against the new coronavirus?',
 '7. can spraying alcohol or chlorine all over your body kill the new coronavirus? #chamber',
 '8. how effective are thermal scanners in detecting people infected with the new coronavirus?',
 '9. can an ultraviolet disinfection lamp kill the new coronavirus?',
 '10. are hand dryers effective in killing the new coronavirus?',
 '11. the new coronavirus cannot be transmitted through mosquito bites.',
 '12. taking a hot bath does not prevent the new coronavirus disease',
 '13. cold weather and snow cannot kill the new coronavirus.',
 '14. covid-19 virus can be transmitted in areas with hot and humid climates',
 '15. drinking alcohol does not protect you against covid-19 and can be dangerous',
 '16. being able to hold your bre

In [12]:
train_data_concatenated = [' '.join(sentence) for sentence in train_data_rebuilt]
dev_data_concatenated = [' '.join(sentence) for sentence in dev_data_rebuilt]
test_data_concatenated = [' '.join(sentence) for sentence in test_data_rebuilt]


In [19]:
print(len(train_data_concatenated))

1566


In [13]:
from transformers import BertModel
from transformers import BertTokenizer

# bert_model = BertModel.from_pretrained('bert-base-uncased')

# print("Done loading BERT model.")

# #load BERT's WordPiece tokenisation model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [21]:
sentence = train_data_concatenated[2]
tokens = tokenizer.tokenize(sentence)
print(tokens)

NameError: name 'tokenizer' is not defined

In [ ]:
tokenizer.convert_tokens_to_ids(tokens)[14]

In [1]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer
import pandas as pd
from torch.utils.data import DataLoader

import torch
from transformers import AutoModel, AutoTokenizer 


In [3]:
BertTokenizer.from_pretrained('bert-base-uncased')

PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [2]:

bertweet = AutoModel.from_pretrained("vinai/bertweet-base")

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [14]:
class TweetDataset(Dataset):

    def __init__(self, series, labels, maxlen):

        #Store the contents of the file in a pandas dataframe
        # if labels==False:
        #   self.df = pd.DataFrame({"sentence":series, "label":["NaN" for i in series]})
        # else:
        self.df = pd.DataFrame({"sentence":series, "label":labels})
        
        #Initialize the BERT tokenizer
        #self.tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'sentence']
        label = self.df.loc[index, 'label']

        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        tokens = [t for t in tokens if t != 'UNK']
        tokens = ['[CLS]'] + tokens + ['[SEP]'] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length
        
        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

In [15]:
train_set = TweetDataset(series = train_data_concatenated, labels = y_train, maxlen = 512)
dev_set = TweetDataset(series = dev_data_concatenated, labels = y_dev, maxlen = 512)
test_set = TweetDataset(series = test_data_concatenated, labels = ["NaN" for i in test_data_concatenated], maxlen = 512)

#Creating intsances of training and development dataloaders
train_loader = DataLoader(train_set, batch_size = 8, num_workers = 2)
dev_loader = DataLoader(dev_set, batch_size = 8, num_workers = 2)
test_loader = DataLoader(test_set, batch_size = 8, num_workers = 2)

In [ ]:
for it, (seq, attn_masks, labels) in enumerate(train_loader):
    print(it)
    print(seq)
    print(attn_masks)
    print(labels)
    break
    #seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)

In [13]:
import torch
import torch.nn as nn
from transformers import BertModel

class RumourDetector(nn.Module):

    def __init__(self):
        super(RumourDetector, self).__init__()
        #Instantiating BERT model object 
        #self.bert_layer = AutoModel.from_pretrained("vinai/bertweet-base")
        
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        #Classification layer
        #input dimension is 768 because [CLS] embedding has a dimension of 768
        #output dimension is 1 because we're working with a binary classification problem
        #self.hidden_layer_1 = nn.Linear(768, 16)
        
        self.cls_layer = nn.Linear(768, 1)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        outputs = self.bert_layer(seq, attention_mask = attn_masks, return_dict=True)
        cont_reps = outputs.last_hidden_state

        #Obtaining the representation of [CLS] head (the first token)
        bert_rep = cont_reps[:, 0]
        
        #hidden_1 = self.hidden_layer_1(bert_rep)
        #print(bert_rep.shape)
        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(bert_rep)

        return logits

In [23]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.8.1+cu101
True


In [14]:
gpu = 0 #gpu ID

print("Creating the sentiment classifier, initialised with pretrained BERT-BASE parameters...")
net = RumourDetector()
net.cuda(gpu) #Enable gpu support for the model
print("Done creating the sentiment classifier.")

Creating the sentiment classifier, initialised with pretrained BERT-BASE parameters...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Done creating the sentiment classifier.


In [15]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [16]:
import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):
        
        net.train()
        print('0')
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            print('1')
            #Obtaining the logits from the model
            logits = net(seq, attn_masks)
            print('2')
            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 100 == 0:
                
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            best_acc = dev_acc
            torch.save(net.state_dict(), 'sstcls_{}.dat'.format(ep))

In [17]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    # print(soft_probs.squeeze())
    # print(np.array(soft_probs.squeeze().cpu()))
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            #print(logits)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1


    return mean_acc / count, mean_loss / count

def get_prediction_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    return soft_probs.squeeze()
def predict(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    prediction = np.array([])
    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks = seq.cuda(gpu), attn_masks.cuda(gpu)
            logits = net(seq, attn_masks)
            #print(logits)
            #print(get_prediction_from_logits(logits, labels).data.cpu().numpy())
            
            prediction = np.concatenate((prediction, get_prediction_from_logits(logits, labels).data.cpu().numpy()), axis=0)
            #print(prediction)
    #predict.data.numpy()


    return prediction.astype('int32')



In [ ]:
num_epoch = 1

#fine-tune the model
train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)

0


In [161]:
net.load_state_dict(torch.load("sstcls_4.dat"))

<All keys matched successfully>

In [ ]:
y_predict = predict(net, criterion, test_loader, gpu)

In [165]:
import pandas as pd
index = [i for i in range(558)]
dataframe = pd.DataFrame({'Id':index,'Predicted':y_predict})
dataframe

,Id,Predicted
0,0,0
1,1,1
2,2,0
3,3,0
4,4,0
...,...,...
553,553,0
554,554,0
555,555,1
556,556,0


In [166]:
dataframe.to_csv("nlp4.csv",index=0)

In [ ]:
# playground

In [60]:
a = np.array([])
b = np.array([1, 2, 3])
np.concatenate((a, b), axis=0)

array([1., 2., 3.])

In [40]:
dev_acc

tensor(0.9704, device='cuda:0')

In [178]:

torch.cuda.empty_cache()

In [179]:
import gc

gc.collect()

10725

In [117]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   13497 MB |   13497 MB |   22203 MB |    8705 MB |\n|       from large pool |   13492 MB |   13492 MB |   22196 MB |    8704 MB |\n|       from small pool |       4 MB |       4 MB |       6 MB |       1 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |   13497 MB |   13497 MB |   22203 MB |    8705 MB |\n|       from large pool |   13492 MB |   13492 MB |

In [113]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May  9 12:39:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W /  70W |  14922MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------